# Vicon CSV → Quantity of Motion (QdM)

This notebook computes:
- 3D marker trajectories from Vicon CSV exports
- Quantity of Motion (QdM) for wrists and head
- Exports results to an Excel summary file

Author: Matys Précloux  
Project: SYNCOGEST

In [18]:
from pathlib import Path
import pandas as pd

from src.vicon_utils import read_vicon_csv, find_xyz_cols, motion_quantity_point

In [3]:

# 1) Vicon CSV Reader

def read_vicon_csv(csv_path: Path) -> pd.DataFrame:
    """
    Parse a Vicon "Trajectories" CSV export.

    Expected layout:
    Row 1: "Trajectories"
    Row 2: sampling frequency
    Row 3: marker names (with blanks that must be forward-filled)
    Row 4: axes (Frame/Sub Frame + X/Y/Z)
    Row 5: units (e.g., mm)
    Row 6+: numeric data

    Returns
    -------
    pd.DataFrame
        DataFrame with flattened column names:
        "<marker>_X", "<marker>_Y", "<marker>_Z"
    """

    with open(csv_path, "r", encoding="utf-8", errors="replace", newline="") as f:
        reader = csv.reader(f)
        header_lines = [next(reader) for _ in range(5)]

    marker_row = header_lines[2]
    axis_row = header_lines[3]

    n = max(len(marker_row), len(axis_row))
    marker_row += [""] * (n - len(marker_row))
    axis_row += [""] * (n - len(axis_row))

    # Forward-fill marker names (Vicon leaves blanks under same marker)
    filled = []
    last = ""
    for m in marker_row:
        m = (m or "").strip()
        if m == "":
            filled.append(last)
        else:
            last = m
            filled.append(last)

    # Build flat column names
    colnames = []
    for m, a in zip(filled, axis_row):
        m = (m or "").strip()
        a = (a or "").strip()

        if a in ["Frame", "Sub Frame"]:
            colnames.append(a)
        elif a in ["X", "Y", "Z"]:
            colnames.append(f"{m}_{a}")
        else:
            colnames.append(m if m else a)

    df = pd.read_csv(csv_path, skiprows=5, header=None, names=colnames, engine="python")
    df = df.dropna(axis=1, how="all")

    return df

In [4]:

# 2) Find X/Y/Z columns for a marker

def find_xyz_cols(cols, token):
    """
    Find X/Y/Z column names for a given marker token.

    Example tokens:
    - "poignet_D"
    - "2poignet_G"
    - "Tempe_D"

    Works even if marker is prefixed:
    e.g. "Patient 1:poignet_D_X"
    """

    pat = re.compile(rf"(?:^|:)\s*{re.escape(token)}_([XYZ])\b", re.IGNORECASE)
    found = {}

    for c in cols:
        c2 = c.replace(" ", "")
        m = pat.search(c2)
        if m:
            axis = m.group(1).upper()
            if axis not in found or len(c) < len(found[axis]):
                found[axis] = c

    return found.get("X"), found.get("Y"), found.get("Z")

In [5]:

# 3) Quantity of Motion (QdM)

def motion_quantity_point(df, X, Y, Z):
    """
    Compute Quantity of Motion (QdM) for a 3D point.

    QdM = sum of Euclidean distances between consecutive frames.

    Returns
    -------
    qdm_mm : float
        Total 3D path length in millimeters.
    n_steps : int
        Number of valid frame-to-frame steps.
    """

    arr = df[[X, Y, Z]].to_numpy(dtype=float)
    d = np.diff(arr, axis=0)
    step = np.sqrt((d**2).sum(axis=1))
    step = step[np.isfinite(step)]

    return float(step.sum()), int(step.size)

In [6]:

# 4) Process one CSV file

def process_one_csv(csv_path: Path):

    df = read_vicon_csv(csv_path)
    cols = list(df.columns)

    out = {"csv": str(csv_path), "file": csv_path.name}

    subjects = {
        "P1": {
            "WR_D": "poignet_D",
            "WR_G": "poignet_G",
            "TP_D": "Tempe_D",
            "TP_G": "Tempe_G",
        },
        "P2": {
            "WR_D": "2poignet_D",
            "WR_G": "2poignet_G",
            "TP_D": "2Tempe_D",
            "TP_G": "2Temps_G",
        }
    }

    for pid, tok in subjects.items():

        # ---- Wrists ----
        for side_key in ["WR_D", "WR_G"]:
            token = tok[side_key]
            X, Y, Z = find_xyz_cols(cols, token)

            if None in (X, Y, Z):
                out[f"{pid}_{token}_error"] = "missing X/Y/Z"
            else:
                q, nsteps = motion_quantity_point(df, X, Y, Z)
                out[f"{pid}_{token}_QDM_mm"] = q
                out[f"{pid}_{token}_nsteps"] = nsteps

        wd = out.get(f"{pid}_{tok['WR_D']}_QDM_mm")
        wg = out.get(f"{pid}_{tok['WR_G']}_QDM_mm")

        if wd is not None and wg is not None:
            out[f"{pid}_QDM_WRISTS_mm"] = wd + wg

        # ---- Head (temples) ----
        for side_key in ["TP_D", "TP_G"]:
            token = tok[side_key]
            X, Y, Z = find_xyz_cols(cols, token)

            if None in (X, Y, Z):
                out[f"{pid}_{token}_error"] = "missing X/Y/Z"
            else:
                q, nsteps = motion_quantity_point(df, X, Y, Z)
                out[f"{pid}_{token}_QDM_mm"] = q
                out[f"{pid}_{token}_nsteps"] = nsteps

        td = out.get(f"{pid}_{tok['TP_D']}_QDM_mm")
        tg = out.get(f"{pid}_{tok['TP_G']}_QDM_mm")

        if td is not None and tg is not None:
            out[f"{pid}_QDM_HEAD_mm"] = td + tg

    return out

In [7]:

# 5) Batch processing

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
ROOT = PROJECT_ROOT / "data" / "test" / "VICON_CSV"
RESULTS_DIR = PROJECT_ROOT / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

csv_files = sorted(ROOT.rglob("*.csv"))
print("CSV found:", len(csv_files))

rows = []

for f in csv_files:
    try:
        rows.append(process_one_csv(f))
    except Exception as e:
        rows.append({"csv": str(f), "file": f.name, "error": repr(e)})

df_out = pd.DataFrame(rows)

out_path = RESULTS_DIR / "vicon_QDM_wrists_head_mm.xlsx"
df_out.to_excel(out_path, index=False)

print("Saved:", out_path.resolve())
df_out.head()

CSV found: 3
Saved: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results/vicon_QDM_wrists_head_mm.xlsx


,csv,file,P1_poignet_D_QDM_mm,P1_poignet_D_nsteps,P1_poignet_G_QDM_mm,P1_poignet_G_nsteps,P1_QDM_WRISTS_mm,P1_Tempe_D_QDM_mm,P1_Tempe_D_nsteps,P1_Tempe_G_QDM_mm,...,P2_2poignet_D_QDM_mm,P2_2poignet_D_nsteps,P2_2poignet_G_QDM_mm,P2_2poignet_G_nsteps,P2_QDM_WRISTS_mm,P2_2Tempe_D_QDM_mm,P2_2Tempe_D_nsteps,P2_2Temps_G_QDM_mm,P2_2Temps_G_nsteps,P2_QDM_HEAD_mm
0,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD01.csv,11257.605164,18011,5086.439843,18011,16344.045006,15748.775325,18011,18437.193954,...,6990.478950,18011,4791.810419,18011,11782.289369,10070.014078,18011,10379.521199,18011,20449.535277
1,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEMID01.csv,8668.166077,17999,6035.964257,17999,14704.130334,13942.428120,17999,14935.841734,...,7473.133557,17999,8656.559835,17999,16129.693392,8852.532002,17999,9113.597368,17999,17966.129370
2,/Users/matysprecloux/Desktop/Master IEAP/Code ...,STANDINGD01.csv,21176.211919,17999,18263.692630,17999,39439.904549,15582.855481,17999,17012.960766,...,23389.330425,17999,22248.213123,17999,45637.543548,13910.023848,17999,13998.875022,17999,27908.898870


In [8]:
# 5) Batch processing

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
ROOT = PROJECT_ROOT / "data" / "raw" / "VICON_CSV"
RESULTS_DIR = PROJECT_ROOT / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

csv_files = sorted(ROOT.rglob("*.csv"))
print("CSV found:", len(csv_files))

rows = []

for f in csv_files:
    try:
        rows.append(process_one_csv(f))
    except Exception as e:
        rows.append({"csv": str(f), "file": f.name, "error": repr(e)})

df_out = pd.DataFrame(rows)

out_path = RESULTS_DIR / "vicon_QDM_wrists_head_mm.xlsx"
df_out.to_excel(out_path, index=False)

print("Saved:", out_path.resolve())
df_out.head()

CSV found: 36
Saved: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results/vicon_QDM_wrists_head_mm.xlsx


,csv,file,P1_poignet_D_QDM_mm,P1_poignet_D_nsteps,P1_poignet_G_QDM_mm,P1_poignet_G_nsteps,P1_QDM_WRISTS_mm,P1_Tempe_D_QDM_mm,P1_Tempe_D_nsteps,P1_Tempe_G_QDM_mm,...,P2_2poignet_D_QDM_mm,P2_2poignet_D_nsteps,P2_2poignet_G_QDM_mm,P2_2poignet_G_nsteps,P2_QDM_WRISTS_mm,P2_2Tempe_D_QDM_mm,P2_2Tempe_D_nsteps,P2_2Temps_G_QDM_mm,P2_2Temps_G_nsteps,P2_QDM_HEAD_mm
0,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD01.csv,11257.605164,18011,5086.439843,18011,16344.045006,15748.775325,18011,18437.193954,...,6990.478950,18011,4791.810419,18011,11782.289369,10070.014078,18011,10379.521199,18011,20449.535277
1,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD02.csv,15422.894012,17999,16779.292373,17999,32202.186385,9725.644869,17999,10354.212811,...,16420.189086,17999,17622.889557,17999,34043.078643,9851.813433,17999,10217.118782,17999,20068.932215
2,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD03.csv,34870.992267,17999,27170.831678,17999,62041.823945,14105.980148,17999,14473.346651,...,19508.827550,17999,6708.991777,17999,26217.819326,8916.973687,17999,9655.490833,17999,18572.464520
3,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD04.csv,2661.923274,17999,3833.346976,17999,6495.270250,4847.753225,17999,4768.206439,...,7680.839058,17999,11250.402281,17999,18931.241339,7442.981553,17999,7080.281923,17999,14523.263476
4,/Users/matysprecloux/Desktop/Master IEAP/Code ...,SEATEDD05.csv,6273.538429,17999,6588.362586,17999,12861.901015,4558.721874,17999,4829.197593,...,2684.099411,17999,5080.767751,17999,7764.867162,5805.341913,17999,5511.189012,17999,11316.530925
